File: ratings.ipynb\
Author: alexkobz\
Date: 07.05.2024\
Task: DSRFU-275

#### Шаг 0. Импорт и получение токена

In [1]:
import asyncio
import requests
import pandas as pd
import numpy as np
import nest_asyncio; nest_asyncio.apply()
from datetime import datetime as dt, date, timedelta
from functions.divide_chunks import divide_chunks
from math import ceil
from functions.get_date import last_day_month, last_day_month_str, last_work_date_month, last_work_date_month_str

# последний день месяца
REPORT_DATE = last_day_month
REPORT_DATE_STR = last_day_month_str
# увеличиваем количество столбцов и строк для просмотра
pd.set_option('display.max_rows', 200, 'display.max_columns', 200)

Unexpected Http Driver Exception


Error HTTPConnectionPool(host='host.docker.internal', port=8124): Max retries exceeded with url: /? (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x166082590>: Failed to resolve 'host.docker.internal' ([Errno 8] nodename nor servname provided, or not known)")) executing HTTP request attempt 1 (http://host.docker.internal:8124)


In [2]:
from rudata.RuDataDF import RuDataDF
from rudata.RuDataRequest import RuDataRequest
RuDataRequest.set_headers()

Getting token started
Getting token finished


#### Шаг 1. Загрузка stage таблиц 

[FintoolReferenceData](https://docs.efir-net.ru/dh2/#/Info/FintoolReferenceData?id=post-fintoolreferencedata) - Получить расширенный справочник по финансовым инструментам.

In [3]:
FintoolReferenceData = RuDataDF("FintoolReferenceData").df
FintoolReferenceData.head()

FintoolReferenceData started
FintoolReferenceData finished. FintoolReferenceData shape (105354, 187)


,facevalue_m,faceftname_m,sumissuevol_m,summarketvol_m,numcoupons_m,nrdcode,regcode_m,regdate_m,regdistdate_m,begdistdate_m,enddistdate_m,endmtydate_m,daysall_m,fullname_nrd,rpregorg_nrd,rpstate_nrd,issuenumber,currentfacevalue_nrd,isqualified_nrd,bondseries,islombardcbr_nrd,numcoupons_nrd,facevalue_nrd,faceftname_nrd,summarketvol_nrd,regcode_nrd,regdate_nrd,regorg_nrd,begdistdate_nrd,daysall_nrd,enddistdate_nrd,endmtydate_nrd,regdistdate_nrd,sumissuevol_nrd,drqty,isincodebase_nrd,nrdcompcode,shqty,summarketvolorig_nrd,issuername_nrd,planmtydate_nrd,firstcoupondate_nrd,depositary_nrd,registrar_nrd,placeprice_nrd,micexlist_nrd,minlot_nrd,formarketbonds_nrd,registraracctype_nrd,registraracctypedate_nrd,facialacc_nrd,acc_open_date_nrd,fraction_nrd,specialized_depo_nrd,cmp_code_nsd_issuer_nrd,cmp_code_nsd_managing_nrd,ratetypenamerus_nrd,ratetypenameeng_nrd,coupontypename_nrd,base_month,base_year,basis_nrd,sectypenamerus_nrd,sectypenameeng_nrd,secformnamerus_nrd,secformnameeng_nrd,sectypenamebr_nrd,secstaterus_nrd,secstateeng_nrd,placementtype_nrd,share_category_name_nrd,share_category_name_en_nrd,dr_category_name_nrd,dr_category_name_en_nrd,company_state_name_nrd,rate_type_id,cp_type_id,coupon_period_base_id,sec_type_id,sec_form_id,sec_state_id,placement_type_id,share_category_id,dr_category_mn,company_state_mn,sector4212u,sna2008,fullname_en_nrd,seniorityname,fintoolid,moex_code,moex_name,fullname,isin144a,isincode,nickname,progfintoolid,regcode,regdate,regorg,amortisedmty,basis,bondstructuralpar,cfi,coupontype,fintooltype,guaranteeamount,guaranteetype,haveoffer,isconvertible,isguaranteed,maturitygroup,securitization,accruedintcalctype,country,couponperyear,faceftname,facevalue,guarantval,havedefault,numcoupons,status,sumissueval,sumissuevol,summarketval,summarketvol,borrowername,borrowerokpo,borrowersector,borroweruid,borrowerinn,borrowercountry,borrowerregionname,issuername,issuerokpo,issuersector,issueruid,issuerinn,issuercountry,issuerregionname,assetssector,numguarantors,begdistdate,begmtydate,daysall,enddistdate,endmtydate,privatedist,regdistdate,securitytype,securitykind,nominaltype,update_time,isregion,nsd_service,raterevisionsperyear,floatratename,issubordinated,note,listing_level,havecovenant,haverepayment,haveindexedfv,numdecimals,couponperiod,ismatched,firstcoupondate,couponrate,iscomplicated,available_for_unqualified,sec_type_br_code,original_country,qualified_test_id,qualified_test_id_tm,base_asset_fintoolid,isqualified,listing_level_spb,have_mm_moex,have_mm_spb,beg_book_date,seniority,tradesite,current_coupontype,profit_type,counter,rn,report_date
0,2.0,2.0,2.0,2.0,2.0,None,0.0,0.0,2.0,2.0,2.0,2.0,2.0,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,520,S2,None,Старшие необеспеченные,226058,None,None,"Соединённые Штаты Америки, 2-летние ноты с пог...",None,US91282CGG06,США-4-250131-об,NaN,None,None,None,0.0,act/act,None,DBFTFR,Постоянный,Облигация,None,Необеспеченные,0.0,0.0,0.0,Краткосрочные,None,1.0,US,2.0,USD,100.00,NaN,0.0,4.0,Погашен,4.300000e+10,4.300000e+08,4.200000e+10,4.200000e+08,Правительство США,None,Государственные,95111.0,None,US,None,Минфин США,None,Государственные,95112.0,None,US,None,None,0.0,2023-01-31T00:00:00,None,731.0,2023-01-31T00:00:00,2025-01-31T00:00:00,0.0,2023-01-31T00:00:00,Гос,Классические,Постоянный,2025-01-31T20:20:27,NaN,Нет информации,2.0,None,NaN,None,NaN,False,False,False,3.0,NaN,True,2023-01-31T00:00:00,0.0,False,0,BON5,US,1000,1000,NaN,False,NaN,0,0,None,416.0,NaN,None,Постоянный,105354,90113,2025-07-31
1,2.0,2.0,2.0,2.0,2.0,None,0.0,0.0,2.0,2.0,2.0,2.0,2.0,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None,

[SecurityRatingTable](https://docs.efir-net.ru/dh2/#/Rating/SecurityRatingTable?id=post-securityratingtable) - Получить рейтинги нескольких бумаг и связанных с ними компаний на заданную дату.

In [4]:
%%time
SecurityRatingTable = RuDataDF("SecurityRatingTable").df
SecurityRatingTable.head()

SecurityRatingTable started
SecurityRatingTable finished. SecurityRatingTable shape (168009, 16)
CPU times: user 1.57 s, sys: 38.5 ms, total: 1.61 s
Wall time: 2.8 s


,id,hid,code_name,id_value,forecast,change,date_award,role,okpo,fininstid,fintoolid,isin,id_rating,code,shortname_rus,report_date
0,0,57,AKRA-Bnd-Nsc,Снят,None,снят,2025-02-17T00:00:00,0,None,6317.0,226920.0,RU000A105UW5,254,RU000A105UW5,Альфа-Банк-002Р-21-боб,2025-07-31
1,0,57,AKRA-Bnd-Nsc,AAA(RU),None,Подтвержден,2025-04-04T00:00:00,0,None,91753.0,226889.0,RU000A105V90,254,RU000A105V90,ЕАБР-003Р-006,2025-07-31
2,0,141,JCR-I-Cr&D-LT-Isc,AAA,Стабильный,Подтвержден,2016-03-15T00:00:00,1,None,95112.0,NaN,None,224,None,Минфин США,2025-07-31
3,0,142,JCR-I-Cr&D-LT-Isc-Icur,AAA,Стабильный,Подтвержден,2023-06-26T00:00:00,1,None,95112.0,NaN,None,249,None,Минфин США,2025-07-31
4,0,143,JCR-I-Cr&D-LT-Isc-Ncur,AAA,Стабильный,Подтвержден,2023-06-26T00:00:00,1,None,95112.0,NaN,None,250,None,Минфин США,2025-07-31


[ExchangeTree](https://docs.efir-net.ru/dh2/#/Info/ExchangeTree?id=post-exchangetree) - Получить иерархию торговых площадок/источников, используемых Интерфакс

In [5]:
ExchangeTree = RuDataDF("ExchangeTree").df
ExchangeTree.head()

ExchangeTree started
ExchangeTree finished. ExchangeTree shape (242, 11)


,id,id_parent,shortname_rus,shortname_eng,fullname_rus,fullname_eng,sort_order,update_date,counter,rn,report_date
0,1,0,РТС,RTS,Московская Биржа - РТС,RTS,12,2020-12-28T17:03:09,242,17,2025-07-31
1,2,1,РТС,RTS,РТС индексы,RTS indexes,2,2020-12-28T17:03:09,242,4,2025-07-31
2,8,1,ФОРТС,FORTS,FORTS,FORTS,8,2020-12-28T17:03:09,242,11,2025-07-31
3,9,0,"ПАО ""СПБ""",PJSC «SPB»,"ПАО ""СПБ""",PJSC «SPB»,20,2020-12-28T17:03:09,242,21,2025-07-31
4,11,0,МБ,MX,Московская Биржа,Moscow Exchange,11,2020-12-28T17:03:09,242,16,2025-07-31


[ListScaleValues](https://docs.efir-net.ru/dh2/#/Rating/ListScaleValues?id=post-listscalevalues) - Список шкал значений рейтингов

In [6]:
ListScaleValues = RuDataDF("ListScaleValues").df
ListScaleValues.head()

ListScaleValues started
ListScaleValues finished. ListScaleValues shape (1760, 9)


,id,scale_id,rating_value,rating_level,fullname_rus,description,aggregation_type,aggregation_level,report_date
0,1,1,AAA(rus),1,Наивысший уровень кредитоспособности,Национальные рейтинги категории «ААА» обознача...,None,NaN,2025-07-31
1,2,1,AA+(rus),2,None,None,None,NaN,2025-07-31
2,3,1,AA(rus),3,Очень высокая кредитоспособность,Национальные рейтинги категории «AA» обозначаю...,None,NaN,2025-07-31
3,4,1,AA-(rus),4,None,None,None,NaN,2025-07-31
4,5,1,A+(rus),5,None,None,None,NaN,2025-07-31


[ListRatings](https://docs.efir-net.ru/dh2/#/Dictionary/ListRatings?id=post-listratings) - Список рейтингов

In [7]:
ListRatings = RuDataDF("ListRatings").df
ListRatings.head()

ListRatings started
ListRatings finished. ListRatings shape (389, 27)


,id,rating_id,code_name,agency,fullname_rus,fullname_eng,for_instrument,for_company,agency_eng,official_name,is_credit,term_type,term_type_name,currency_type,currency_type_name,scale_type,scale_type_name,is_archive,is_accred_br,doc_511p,doc_421p,doc_180i,agency_id,usedInAgg,scale_id,aggregation_type,report_date
0,4,23,MDS-I-Corp-LT-Isc-Icur,Мудиз,"Корпоративный рейтинг эмитента, долгосрочный, ...","Сorporate family rating, long-term, internatio...",0,1,Moody's,LT Corporate Family Ratings (Foreign),1,L,Долгосрочный,F,Иностранная,I,Международная,0,1.0,1,1,1,2,1,5,BIG3,2025-07-31
1,5,83,MDS-I-Corp-LT-Isc-Ncur,Мудиз,"Корпоративный рейтинг эмитента, долгосрочный, ...","Сorporate family rating, long-term, internatio...",0,1,Moody's,LT Corporate Family Ratings (Domestic),1,L,Долгосрочный,L,Локальная,I,Международная,0,1.0,1,1,1,2,1,5,BIG3,2025-07-31
2,6,116,MIRA-I-Corp-LT-Nsc,Мудис Интерфакс,"Корпоративный рейтинг эмитента, долгосрочный, ...","Сorporate family rating, long-term, national s...",0,1,Moody's Interfax,NSR LT Corporate Family Ratings (Domestic),1,L,Долгосрочный,L,Локальная,N,Национальная,1,NaN,0,0,0,7,0,2,None,2025-07-31
3,7,129,MDS-I-Corp-LT-UAsc,Мудиз,"Корпоративный рейтинг эмитента, долгосрочный, ...","Сorporate family rating, long-term, ukrainian ...",0,1,Moody's,LT Corporate Family Ratings (ukr),1,L,Долгосрочный,L,Локальная,U,Укр.национальная,0,1.0,0,0,0,2,0,23,None,2025-07-31
4,8,234,EXP-Bnd-Isc,Эксперт РА,"Рейтинг надежности облигационных займов, между...","Reliability ratings for bond loans, internatio...",1,0,Expert RA,Рейтинги надежности облигационных займов,1,L,Долгосрочный,N,Не определена,I,Международная,1,1.0,0,0,0,4,0,32,None,2025-07-31


[Emitents](https://docs.efir-net.ru/dh2/#/Info/Emitents?id=post-emitents) - Получить краткий справочник по эмитентам.

In [8]:
Emitents = RuDataDF("Emitents").df
Emitents.head()

Emitents started
Emitents finished. Emitents shape (42512, 92)


,id_emitent,fininstid,shortname_rus,shortname_eng,inn,okpo,ogrn,code,sector,legal_address,phone,www,update_date,fullname_rus_nrd,fullname_eng_nrd,shortname_rus_nrd,shortname_eng_nrd,company_type_short_name,state_reg_num,state_reg_date,state_reg_name,egrul_date,egrul_organ_name,country,lei_code,bik,fax,e_mail,credit_cmp,is_bank_4_non_resident,swift,country_oksm,country_oksm_cbr,country_name_rus,country_name_eng,region_inn,region_soato,region_name,fullname_rus,sparkid,br_fcsm_reg_code,capital,capital_unit,okved,oecd_lvl,oktmo,okato,post_address,okopf,okogu,have_rating,have_risk,is_finorg,is_control_by_finorg,is_gov_or_cb,on_behalf_of_state,is_subjectrf,is_cis_reg,issuer_nrd,tin,kpp,state_reg_number,market_id_nrd,market_name_rus_nrd,market_name_eng_nrd,sp_rx_entity_id,sp_shortname,mds_shortname,mds_org_id,fch_id,fch_shortname,is_monopoly,is_strategic,ifo_list,isregion,sic,sector4212u,sna2008,reg_code,reg_date,reg_org,note,primary_reg_date,inn_fle,other_tin,other_tin_name,okfs_id,okfs_name,is_branch,counter,rn,report_date
0,24128,7962,Элара,ELARA,2129017646,24322961,1022101269123,ELAR,Машиностроение,"428017,Чувашская Республика - Чувашия, г. Чебо...",(903)3582649; (8352)425303; (8352)453650,www.elara.ru,2022-11-24T16:45:21.469925,None,None,None,None,НАО,None,None,None,None,None,RUS,None,None,None,audit@elara.ru,None,None,None,643.0,643.0,РОССИЯ,Russia,21,97,Чувашская Республика,"""Научно-производственный комплекс ""ЭЛАРА"" имен...",212225.0,55813-D,105368.0,RUB,26.51.1,7,97701000001,97401000000,"428017,Чувашская Республика - Чувашия, г. Чебо...",12267,4210008,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,None,None,213001001,None,NaN,None,None,None,None,None,None,None,None,0,1,None,0.0,3810,None,S112,1022101269123,2002-08-16T00:00:00,Администрация Московского Района Города Чебокс...,None,1992-05-13T00:00:00,None,None,ДопИНН,16,Частная собственность,0,42512,767,2025-07-31
1,24129,66181,Элсиб,NPO ELSIB,5403102702,5757937,1025401300748,ELIB,Машиностроение,"630088,Новосибирская обл., г. Новосибирск, ул....",(383)2278150; (383)2278152; (383)2278153,http://www.elsib.ru/,2025-01-29T14:51:43,"Научно-производственное объединение ""ЭЛСИБ"" пу...",ELSIB,"НПО ""ЭЛСИБ"" ПАО",ELSIB,ПАО,ГР 1110,1994-05-11T00:00:00,Новосибирская городская регистрационная палата...,2002-09-12T00:00:00,Инспекция Министерства РФ по налогам и сборам ...,RUS,None,None,(383) 298-92-94,elsib@elsib.ru,0,0,None,643.0,643.0,РОССИЯ,Russia,54,50,Новосибирская область,"Научно-производственное объединение ""ЭЛСИБ"", ПАО",80607.0,10917-F,112000000.0,RUB,27.11.1,7,50701000001,50401374000,"630088,Новосибирская обл., г. Новосибирск, ул....",12247,4210008,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ELIB,None,540301001,ГР 1110,2.0,Корпоративный,corporate,None,None,None,None,None,None,0,0,None,0.0,3510,None,S112,1025401300748,None,None,None,1992-12-25T00:00:00,None,None,ДопИНН,16,Частная собственность,0,42512,1135,2025-07-31
2,24131,98136,Элпа НИИ,Elpa,7735064772,7628061,1027739098760,ELPA,Технологии,"124460, г. Москва, г. Зеленоград, проспект Пан...",(903)7114552; (495)5301302; (499)5351310,www.elpapiezo.ru,2025-01-29T14:51:44,"Акционерное общество ""Научно-исследовательский...","""Elpa"" Research institute and pilot line"" (joi...","АО ""НИИ ""Элпа""","""Elpa"" Research institute"" (jsc)",АО,None,None,None,2002-08-21T00:00:00,Межрайонная инспекция МНС России № 39 по г. Мо...,RUS,None,None,(499) 710-13-02,elpalaw@elpapiezo.ru,0,0,None,643.0,643.0,РОССИЯ,Russia,77,45,г.Москва,"""Научно-исследовательский институт ""Элпа"" с оп...",84848.0,02540-A,20543500.0,RUB,72.19,7,45331000000,45272562000,"124460, г. Москва, г. Зеленоград, проспект Пан...",12267,4210008,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ELPA,None,773501001,None,2.0,Корпоративный,corporate,None,None,None,None,None,None,0,0,None,0.0,8730,None,S112,1027739098760,None,None,None,1993-06-10T00:00:00,None,None,ДопИНН,16,Частная собственность,0,42512,12104,2025-07-31
3,24132,97866,Электропривод,Elektroprivod,7708012528,216390,1027700022228,ELPD,Машиностроение,"107078

[OfferorsGuarants](https://docs.efir-net.ru/dh2/#/Bond/OfferorsGuarants?id=post-offerorsguarants) - Возвращает список гарантов/оферентов для инструмента

In [9]:
OfferorsGuarants = RuDataDF("OfferorsGuarants").df
OfferorsGuarants.head()

OfferorsGuarants started
OfferorsGuarants finished. OfferorsGuarants shape (5541, 6)


,fintoolId,fininstid_guarant,fininstid_offeror,counter,rn,report_date
0,30319,6805,NaN,5541,1,2025-07-31
1,36324,6874,NaN,5541,2,2025-07-31
2,36325,6874,NaN,5541,3,2025-07-31
3,53479,6874,NaN,5541,4,2025-07-31
4,56915,88767,NaN,5541,5,2025-07-31


[CompanyRatingsTable](https://docs.efir-net.ru/dh2/#/Rating/CompanyRatingsTable?id=post-companyratingstable) - Получить рейтинги нескольких компаний на заданную дату.

In [10]:
CompanyRatingsTable = RuDataDF("CompanyRatingsTable").df
CompanyRatingsTable.head()

CompanyRatingsTable started
CompanyRatingsTable finished. CompanyRatingsTable shape (18851, 14)


,name_ra,dt,last,last_dt,change,forecast,advanced,prev,prev_dt,id_rating,code,code_type,company_name,report_date
0,Рейтинг отозван,2016-03-18T00:00:00,Снят,2016-03-18T00:00:00,снят,None,Основной,Aa3.ru,2016-03-10T00:00:00,6,6685,FININSTID,КАМАЗ,2025-07-31
1,Рейтинг отозван,2011-04-01T00:00:00,Снят,2011-04-01T00:00:00,снят,None,Основной,A2.ru,2007-04-05T00:00:00,6,86633,FININSTID,ЮТК,2025-07-31
2,Рейтинг отозван,2016-03-18T00:00:00,Снят,2016-03-18T00:00:00,снят,None,Основной,Aa1.ru,2016-01-22T00:00:00,6,6391,FININSTID,Башнефть,2025-07-31
3,Рейтинг отозван,2008-10-02T00:00:00,Снят,2008-10-02T00:00:00,снят,None,Основной,A2.ru,2008-05-06T00:00:00,6,7482,FININSTID,ЧТПЗ,2025-07-31
4,Рейтинг отозван,2016-03-18T00:00:00,Снят,2016-03-18T00:00:00,снят,None,Основной,Aa1.ru,2016-01-22T00:00:00,6,7319,FININSTID,Татнефть,2025-07-31


#### Шаг 2. Предобработка таблиц до экспорта в excel

In [11]:
#добавляем информацию из справочника в нашу таблицу с компаниями
ExtendedSecurityRatingTable = SecurityRatingTable.merge(ListRatings[['rating_id','agency','official_name','term_type_name','scale_type_name','scale_id']], how='left', left_on='id_rating', right_on='rating_id')
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable.merge(ListScaleValues, how='left', left_on=['scale_id', 'id_value'], right_on=['scale_id', 'rating_value'])
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[[
    'id_x', 'hid', 'code_name', 'id_value', 'forecast', 'change',
    'date_award', 'role', 'okpo', 'fininstid', 'fintoolid', 'isin',
    'id_rating', 'code', 'shortname_rus', 'rating_id', 'agency',
    'official_name', 'term_type_name', 'scale_type_name', 'scale_id',
    'aggregation_level', 'aggregation_type']].rename(
    columns={
    'id_x': 'id', 'aggregation_level': 'level', 'aggregation_type': 'agg_type'
    }
)
agencies = ['АКРА', 'Эксперт РА', 'НКР', 'НРА'
           # ]
            , 'Мудис Интерфакс']
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[ExtendedSecurityRatingTable['agency'].isin(agencies)]
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[ExtendedSecurityRatingTable['term_type_name'] != 'Краткосрочный']
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[~ExtendedSecurityRatingTable['id_value'].isin(['****','Приостановлен','Снят'])]
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[~ExtendedSecurityRatingTable['isin'].isna()]


In [12]:
akra_list = [
'Structured Finance Instrument Rating',
 'Рейтинг эмиссии, международная шкала',
 'Старший необеспеченный рейтинг эмиссии',
 'Старший обеспеченный рейтинг эмиссии',
 'Субординированный рейтинг эмиссии'
]
expert_list = [
    'Structured finance instrument rating',
    'Кредитный рейтинг долговых инструментов'
]
nkr_list = [
   # 'Кредитный рейтинг долговых обязательств'
]
nra_list = [
 #   'Кредитный рейтинг отдельных выпусков облигаций',
  #  'Рейтинг кредитоспособности выпусков облигаций Индивидуальный'
]
moodys_list = [
    'NSR Senior Unsecured (Domestic)',
 'Senior Unsecured (Domestic); Long-Term Corporate Obligation Rating (Domestic); Structured Finance Long-Term Rating (Domestic)',
 'Senior Unsecured (Foreign); Long-Term Corporate Obligation Rating (Foreign); Structured Finance Long-Term Rating (Foreign)',
 'Subordinate (Domestic)',
 'Subordinate (Foreign)'
]

ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[
((ExtendedSecurityRatingTable['agency']=='АКРА') & (ExtendedSecurityRatingTable['official_name'].isin(akra_list))) |
((ExtendedSecurityRatingTable['agency']=='Эксперт РА') & (ExtendedSecurityRatingTable['official_name'].isin(expert_list))) |
((ExtendedSecurityRatingTable['agency']=='НКР') & (ExtendedSecurityRatingTable['official_name'].isin(nkr_list))) |
((ExtendedSecurityRatingTable['agency']=='НРА') & (ExtendedSecurityRatingTable['official_name'].isin(nra_list))) |
((ExtendedSecurityRatingTable['agency']=='Мудис Интерфакс') & (ExtendedSecurityRatingTable['official_name'].isin(moodys_list)))
 ]

ExtendedSecurityRatingTable = ExtendedSecurityRatingTable\
.dropna(subset=['isin'])\
.drop_duplicates(subset=["isin", "agency"])\
.reset_index(drop=True)

In [13]:
ExtendedSecurityRatingTablePivot = ExtendedSecurityRatingTable.drop_duplicates().pivot(index='isin', columns='agency', values='id_value')
ExtendedSecurityRatingTablePivot.columns = pd.MultiIndex.from_tuples([('Эмиссия', i) for i in ExtendedSecurityRatingTablePivot.columns])

In [14]:
TransformedFintoolReferenceData = FintoolReferenceData.copy()[['isincode', 'nickname', 'fintooltype', 'country', 'faceftname', 'fintoolid',
                   'issuername', 'issuercountry', 'issuerinn', 'issueruid',
                   'borrowername', 'borrowercountry', 'borrowerinn', 'borroweruid',
                   'isguaranteed', 'guaranteetype', 'guaranteeamount', 'guarantval',
                   ]].drop_duplicates(subset=['isincode'])
TransformedFintoolReferenceData[['fintoolid', 'issueruid', 'borroweruid']] = TransformedFintoolReferenceData[['fintoolid', 'issueruid', 'borroweruid']].fillna(0).astype('int')
TransformedFintoolReferenceData = TransformedFintoolReferenceData.merge(OfferorsGuarants[['fintoolId','fininstid_guarant']], how='left', left_on='fintoolid', right_on='fintoolId')
ids = list(TransformedFintoolReferenceData.loc[~TransformedFintoolReferenceData['fininstid_guarant'].isna(),'fininstid_guarant'].astype('int'))
TransformedEmitents = Emitents[Emitents.fininstid.isin(ids)][['shortname_rus','country','inn','fininstid']]
TransformedEmitents.columns = ['name_guarant','country_guarant','inn_guarant','fininstid']
TransformedFintoolReferenceData = TransformedFintoolReferenceData.merge(TransformedEmitents, how='left', left_on='fininstid_guarant', right_on='fininstid')
TransformedFintoolReferenceData = TransformedFintoolReferenceData.drop(columns=['fintoolId', 'fininstid'])
TransformedFintoolReferenceData = TransformedFintoolReferenceData.set_index('isincode').rename_axis('ISIN')
TransformedFintoolReferenceData = TransformedFintoolReferenceData.rename(
    columns={
        'nickname': ('Эмиссия','Название'),
        'fintooltype': ('Эмиссия','Тип'),
        'country': ('Эмиссия','Страна'), 
        'faceftname': ('Эмиссия','Валюта'), 
        'fintoolid': ('Эмиссия','Id RuData'),
        'issuername': ('Эмитент','Название'), 
        'issuercountry': ('Эмитент','Страна'), 
        'issuerinn': ('Эмитент','ИНН'), 
        'issueruid': ('Эмитент','Id RuData'), 
        'borrowername': ('Заемщик','Название'),
        'borrowercountry': ('Заемщик','Страна'), 
        'borrowerinn': ('Заемщик','ИНН'), 
        'borroweruid': ('Заемщик','Id RuData'), 
        'isguaranteed': ('Гарант','Есть гарантия'),
        'guaranteetype': ('Гарант','Вид гарантии'), 
        'guaranteeamount': ('Гарант','Полнота гарантии'), 
        'guarantval': ('Гарант','Гарантированная сумма'), 
        'fininstid_guarant': ('Гарант','Id RuData'),
        'name_guarant': ('Гарант','Название'), 
        'country_guarant': ('Гарант','Страна'), 
        'inn_guarant': ('Гарант','ИНН')
    }
)
TransformedFintoolReferenceData.columns = pd.MultiIndex.from_tuples(TransformedFintoolReferenceData.columns)

In [15]:
ExtendedFintoolReferenceData = TransformedFintoolReferenceData.join(ExtendedSecurityRatingTablePivot)
new_cols = ExtendedFintoolReferenceData.columns.reindex(['Эмиссия','Эмитент','Заемщик','Гарант'], level=0)
ExtendedFintoolReferenceData = ExtendedFintoolReferenceData.reindex(columns=new_cols[0])

In [16]:
ExtendedCompanyRatingsTable = CompanyRatingsTable.merge(ListRatings[['id','agency','official_name','term_type_name','scale_type_name','scale_id']], how='left', left_on='id_rating', right_on='id')
##добавляем уровень рейтинг
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.merge(ListScaleValues, how='left', left_on=['scale_id', 'last'], right_on=['scale_id', 'rating_value'])

In [17]:
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[['name_ra', 'dt', 'last', 'last_dt', 'change', 'forecast', 'advanced',
           'prev', 'prev_dt', 'id_rating', 'code', 'code_type', 'company_name',
           'id_x', 'agency', 'official_name', 'term_type_name', 'scale_type_name',
           'scale_id', 'aggregation_level', 'aggregation_type']].rename(columns={'id_x': 'id', 'aggregation_level': 'level', 'aggregation_type': 'agg_type'})
ExtendedCompanyRatingsTable['id_rating'] = ExtendedCompanyRatingsTable['id_rating'].fillna(0)

In [18]:
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[ExtendedCompanyRatingsTable['agency'].isin(['АКРА','Эксперт РА','НКР','НРА','Мудис Интерфакс'])]
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[~ExtendedCompanyRatingsTable['last'].isin(['****','Приостановлен','Снят'])]
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[ExtendedCompanyRatingsTable['term_type_name'] != 'Краткосрочный']

In [19]:
akra_list=[
 'Рейтинг эмитентов',
 'Long-term credit rating',
 'Long-term local currency credit rating',
 'Long-term foreign currency credit rating',
]
expert_list=[
 'Рейтинг кредитоспособности организаций, осуществляющих микрофинансовую деятельность (Республика Казахстан)',
 'Рейтинг кредитоспособности проектных компаний',
 'Рейтинги кредитоспособности банка Республики Беларусь',
 'Рейтинги кредитоспособности банков',
 'Рейтинги кредитоспособности депозитариев',
 'Рейтинги кредитоспособности лизинговых компаний',
 'Рейтинги кредитоспособности микрофинансовых организаций',
 'Рейтинги кредитоспособности нефинансовых компаний',
 'Рейтинги кредитоспособности нефинансовых компаний Республики Беларусь',
 'Рейтинги кредитоспособности нефинансовых компаний Республики Казахстан',
 'Рейтинги кредитоспособности субъектов федерации и муниципальных образований',
 'Рейтинги кредитоспособности факторинговых компаний',
 'Рейтинги кредитоспособности финансовых компаний',
 'Рейтинги кредитоспособности холдинговых компаний',
 'Рейтинги надежности компаний по страхованию жизни',
 'Рейтинги финансовой надежности негосударственных пенсионных фондов',
 'Рейтинги финансовой надежности страховых компаний'
]
nkr_list=[
 'Кредитный рейтинг НКР по Национальной шкале, Банки',
 'Кредитный рейтинг НКР по Национальной шкале, Нефинансовые компании'
]
nra_list=[
 'Кредитный рейтинг кредитных организаций',
 'Кредитный рейтинг нефинансовых компаний, national scale',
]
moodys_list=[
 'BACKED LT Bank Deposits',
 'BACKED LT Bank Deposits (Domestic)',
 'BACKED LT Bank Deposits (Foreign)',
 'BACKED LT Counterparty Risk Rating',
 'BACKED LT Counterparty Risk Rating (Domestic)',
 'BACKED LT Counterparty Risk Rating (Foreign)',
 'BACKED LT Issuer Rating',
 'BACKED LT Issuer Rating (Domestic)',
 'BACKED LT Issuer Rating (Foreign)',
 'BACKED Senior Unsecured (Domestic)',
 'BACKED Senior Unsecured (Foreign)',
 'Issuer Ratings',
 'LT Bank Deposits',
 'LT Bank Deposits (Domestic)',
 'LT Bank Deposits (Foreign)',
 'LT Bank Deposits (kaz)',
 'LT Bank Deposits (ukr)',
 'LT Corporate Family Ratings',
 'LT Corporate Family Ratings (Domestic)',
 'LT Corporate Family Ratings (Foreign)',
 'LT Corporate Family Ratings (ukr)',
 'LT Counterparty Risk Rating (Domestic)',
 'LT Counterparty Risk Rating (Foreign)',
 'LT Insurance Financial Strength',
 'LT Insurance Financial Strength (Domestic)',
 'LT Insurance Financial Strength (Foreign)',
 'LT Issuer Rating',
 'LT Issuer Rating (Domestic)',
 'LT Issuer Rating (Foreign)',
 'Senior Unsecured',
 'Senior Unsecured (Domestic)',
 'Senior Unsecured (Foreign)',
]

ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[
((ExtendedCompanyRatingsTable['agency']=='АКРА') & (ExtendedCompanyRatingsTable['official_name'].isin(akra_list))) |
((ExtendedCompanyRatingsTable['agency']=='Эксперт РА') & (ExtendedCompanyRatingsTable['official_name'].isin(expert_list))) |
((ExtendedCompanyRatingsTable['agency']=='НКР') & (ExtendedCompanyRatingsTable['official_name'].isin(nkr_list))) |
((ExtendedCompanyRatingsTable['agency']=='НРА') & (ExtendedCompanyRatingsTable['official_name'].isin(nra_list))) |
((ExtendedCompanyRatingsTable['agency']=='Мудис Интерфакс') & (ExtendedCompanyRatingsTable['official_name'].isin(moodys_list)))
]

In [20]:
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.dropna(subset=['code'])
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.reset_index(drop=True)

In [21]:
for n, g in ExtendedCompanyRatingsTable.groupby(['code', 'agency']):
    if g.shape[0] > 1: 
        try:
            m = g['level'].idxmin(skipna=False)
            if pd.isna(m):
                m = g.index[0]
            to_drop = g.index.drop(m)
            ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.drop(index=to_drop)
        except:
            print(g)
            raise
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.dropna(subset=['code'])
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.reset_index(drop=True)
ExtendedCompanyRatingsTable['code'] = ExtendedCompanyRatingsTable['code'].astype(int)

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_1552/795657297.py:4: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  m = g['level'].idxmin(skipna=False)
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_1552/795657297.py:4: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  m = g['level'].idxmin(skipna=False)
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_1552/795657297.py:4: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  m = g['level'].idxmin(skipna=False)
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_1552/795657297.py:4: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is depre

In [22]:
ExtendedCompanyRatingsPivot = ExtendedCompanyRatingsTable.pivot(index='code', columns='agency', values='last')

In [23]:
df1 = ExtendedFintoolReferenceData.copy()
df1.columns =df1.columns.to_flat_index()
for k in ['Эмитент','Заемщик','Гарант']:
    df2 = ExtendedCompanyRatingsPivot.copy()
    df2.columns = [(k, i) for i in df2.columns]
    df1 = df1.merge(df2, how='left', left_on=[(k, 'Id RuData')], right_index=True)
df1.columns = pd.MultiIndex.from_tuples(df1.columns)
new_cols = df1.columns.reindex(['Эмиссия', 'Эмитент', 'Заемщик', 'Гарант'], level=0)
df1 = df1.reindex(columns=new_cols[0])
ExtendedFintoolReferenceData = df1

In [24]:
df = Emitents[Emitents['fininstid'].isin(ExtendedCompanyRatingsPivot.index)]
CompanyRatings = df.merge(ExtendedCompanyRatingsPivot, left_on='fininstid', right_index=True, how='left')

#### Шаг 3. Рейтинги облигаций из фактовой таблицы бумаг FintoolReferenceData. Рейтинги облигаций из фактовой таблицы компаний CompanyRatingsTable

In [25]:
BondRatings = ExtendedFintoolReferenceData['Эмиссия'].reset_index()
id_vars = BondRatings.columns[:6]
value_vars = BondRatings.columns[6:]
meltBonds = pd.melt(BondRatings, id_vars=id_vars, value_vars=value_vars, var_name='agency', value_name='rate')
meltBonds = meltBonds[~meltBonds.rate.isna()].sort_values('Id RuData')
meltBonds = meltBonds[meltBonds["Тип"] == 'Облигация']

In [26]:
CompanyRatings = pd.concat([
    CompanyRatings[["id_emitent", "shortname_rus", "inn", "update_date", "country_oksm", "have_rating", "have_risk", "tin", "fininstid", "reg_code", "lei_code"]]
    , CompanyRatings.iloc[:, CompanyRatings.columns.get_loc('rn')+1:]
], axis=1)
id_vars = CompanyRatings.columns[:11]
value_vars = CompanyRatings.columns[11:]
meltCompanies = pd.melt(CompanyRatings, id_vars=id_vars, value_vars=value_vars, var_name='agency', value_name='rate')
meltCompanies = meltCompanies[meltCompanies['rate'].notnull()].sort_values('id_emitent')
meltCompanies['inn'] = np.where(meltCompanies['country_oksm'] == 643, meltCompanies['inn'], meltCompanies['tin'] )
meltCompanies['country_oksm'] = meltCompanies['country_oksm'].astype(np.int32)
meltCompanies['type'] = "ЮЛ"

In [27]:
meltBonds.rename(columns={
    "Id RuData": "id_emitent",
    "Название": "company_name", 
    "Тип": "rate_type",
    "ISIN": "inn/isin",
    "Страна": "country"
    }, inplace=True)
meltCompanies.rename(columns={
    "id_rudata": "id_emitent",
    "shortname_rus": "company_name", 
    "type": "rate_type",
    "inn": "inn/isin",
    "country_oksm": "country_code"
    }, inplace=True)
meltBonds["fininstid"] = np.nan
meltBonds["reg_code"] = np.nan
meltBonds["lei_code"] = np.nan
meltBonds["report_date"] = pd.to_datetime(REPORT_DATE)
meltCompanies["report_date"] = pd.to_datetime(REPORT_DATE)

Справочник кодов стран ОКСМ

In [29]:
countries = RuDataDF.client.query_df(
    """
    SELECT DISTINCT country, alpha2
    FROM countries
    """)
countries["country"] = countries["country"].astype(np.int32)
co = countries.set_index("alpha2").country.to_dict()
meltBonds["country_code"] = meltBonds["country"].map(co)

#### Шаг 4. Получаем результирующий результат по рейтингам облигаций и компаний и экспорт в excel

In [30]:
meltBonds = meltBonds[["id_emitent", "fininstid", "company_name", "inn/isin", "country_code", "agency", "rate", "report_date", "rate_type", "reg_code", "lei_code"]]
meltCompanies = meltCompanies[["id_emitent", "fininstid", "company_name", "inn/isin", "country_code", "agency", "rate", "report_date", "rate_type", "reg_code", "lei_code"]]
ResultRatings = pd.concat([meltBonds, meltCompanies], axis=0)
with pd.ExcelWriter('data/Output/ratings.xlsx', datetime_format='dd.mm.yyyy') as writer:
    ResultRatings.to_excel(writer, index=False)

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_1552/4284157848.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ResultRatings = pd.concat([meltBonds, meltCompanies], axis=0)


In [31]:
from functions.send_email import main
main("ratings.xlsx")